# Extract the parameters of the train images

## Importation of the needed libraries

In [1]:
from scipy.spatial import distance as dist
from imutils import perspective
from imutils import contours
import argparse
import numpy as np
import imutils
import cv2
from os import listdir
from os.path import isfile, join
import pandas as pd  
import datetime 

## Definition of some functions to use on the images

In [3]:
def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

def show_image(title, image, destroy_all=True):
    cv2.imshow(title, image)
    cv2.waitKey(0)
    if destroy_all:
        cv2.destroyAllWindows()


def giveLength_WidthRelation(image,width):
    cnts = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    print("Total number of contours are: ", len(cnts))

    (cnts, _) = contours.sort_contours(cnts)
    pixelPerMetric = None


    max_area = cv2.contourArea(cnts[0])
    contour_max = cnts[0]
    for cn in cnts:
        if cv2.contourArea(cn)>max_area:
            max_area = cv2.contourArea(cn)
            contour_max = cn
        

    orig = image.copy()
    box = cv2.minAreaRect(contour_max)
    box = cv2.cv.BoxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
    box = np.array(box, dtype="int")

    box = perspective.order_points(box)
    cv2.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 2)

    for (x, y) in box:
        cv2.circle(orig, (int(x), int(y)), 5, (0, 0, 255), -1)


    (tl, tr, br, bl) = box
    (tltrX, tltrY) = midpoint(tl, tr)
    (blbrX, blbrY) = midpoint(bl, br)
    (tlblX, tlblY) = midpoint(tl, bl)
    (trbrX, trbrY) = midpoint(tr, br)

    cv2.circle(orig, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
    cv2.circle(orig, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

    cv2.line(orig, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)), (255, 0, 255), 2)
    cv2.line(orig, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)), (255, 0, 255), 2)

    dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
    dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

    if pixelPerMetric is None:
        pixelPerMetric = dB / width

    dimA = dA / pixelPerMetric
    dimB = dB / pixelPerMetric

    if (dimA>= dimB):
        Lange = dimA
        Breite = dimB
    else:
        Lange = dimB
        Breite = dimA

    if (dimB==0 or dimA==0):
        return (0,0)
    else:  
        return (Lange/Breite,len(cnts))



## Do the extraction

In [6]:
mypath_foeffner='Meine_Bilder/Flaschenoeffner'
onlyfiles_foeffner = [ f for f in listdir(mypath_foeffner) if isfile(join(mypath_foeffner,f)) ]
mypath_dreher='Meine_Bilder/Korkenzieher'
onlyfiles_dreher = [ f for f in listdir(mypath_dreher) if isfile(join(mypath_dreher,f)) ]

print(onlyfiles_dreher[0])
print(onlyfiles_foeffner[0])

nKamm = len(onlyfiles_foeffner)
nBuerste = len(onlyfiles_dreher)

nDataForEachClass = min(nKamm,nBuerste)
nData = 2*nDataForEachClass

#Creation of the dataFrame

df = pd.DataFrame(columns=['Klasse','Lange/Breite','Anzahl_an_Konturen_gefunden_1','Anzahl_an_Konturen_gefunden_2'], index=range(nData))
print(df)


for i in range(nDataForEachClass):

    print('Analyse image :'+ str(i))
    imKamm = cv2.imread( join(mypath_foeffner
,onlyfiles_foeffner[i]) )
    imBuerste = cv2.imread( join(mypath_dreher,onlyfiles_dreher[i]) )
    cv2.waitKey()

    #Kamm image 

    grayK = cv2.cvtColor(imKamm, cv2.COLOR_BGR2GRAY)
    gray_filteredK = cv2.bilateralFilter(grayK,7,30,30)

    edged_filteredK=cv2.Canny(gray_filteredK,50,120)

    if i == 3 or i == 4:
        cv2.imshow('dst1',edged_filteredK)
        cv2.waitKey()
        cv2.destroyAllWindows()

    edged_filteredK = cv2.dilate(edged_filteredK, None, iterations=1)


    if i == 3 or i == 4:
        cv2.imshow('dst2',edged_filteredK)
        cv2.waitKey()
        cv2.destroyAllWindows()

    edged_filteredK = cv2.erode(edged_filteredK, None, iterations=1)

    if i == 3 or i == 4:
        cv2.imshow('dst3',edged_filteredK)
        cv2.waitKey()
        cv2.destroyAllWindows()

    valueLength_WidthK = giveLength_WidthRelation(edged_filteredK,2)

    #With this filtered image we take 2 parameters, the ratio of the length by the width and the number of contours found


    #Now we take the number of countours found with other filters on the image
    #We don't know yet which will be the best, we have to try

    #convert into grayscale
    gray2K = cv2.cvtColor(imKamm, cv2.COLOR_BGR2GRAY)

    #blur image
    blur2K = cv2.GaussianBlur(gray2K, (9,9), 1)
    
    #set threshold of blur image
    _, thresh2K = cv2.threshold(blur2K, 127, 255, cv2.THRESH_BINARY)

    #find contours
    contours2K, _ = cv2.findContours(
        thresh2K, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    nCounts2K = len(contours2K)


    df.loc[i]=[0,valueLength_WidthK[0],valueLength_WidthK[1],nCounts2K]    # O für Kamm in der Spalte Klasse


     #Buerste image 

    grayB = cv2.cvtColor(imBuerste, cv2.COLOR_BGR2GRAY)
    gray_filteredB = cv2.bilateralFilter(grayB,7,30,30)

    edged_filteredB=cv2.Canny(gray_filteredB,50,120)

    edged_filteredB = cv2.dilate(edged_filteredB, None, iterations=1)
    edged_filteredB = cv2.erode(edged_filteredB, None, iterations=1)

    valueLength_WidthB = giveLength_WidthRelation(edged_filteredB,2)

    #Now we take the number of countours found with other filters on the image
    #We don't know yet which will be the best, we have to try

    #convert into grayscale
    gray2B = cv2.cvtColor(imBuerste, cv2.COLOR_BGR2GRAY)

    #blur image
    blur2B = cv2.GaussianBlur(gray2B, (9,9), 1)
    
    #set threshold of blur image
    _, thresh2B = cv2.threshold(blur2B, 127, 255, cv2.THRESH_BINARY)

    #find contours
    contours2B, _ = cv2.findContours(
        thresh2B, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
    )
    nCounts2B = len(contours2B)

    df.loc[nDataForEachClass+i]=[1,valueLength_WidthB[0],valueLength_WidthB[1],nCounts2B] # 1 für Bürste in der Spalte Klasse 

print(df)
cv2.waitKey()
cv2.destroyAllWindows()

WhatsApp Image 2021-12-29 at 17.56.00 (1).jpeg
WhatsApp Image 2021-12-29 at 17.55.16 (1).jpeg
   Klasse Lange/Breite Anzahl_an_Konturen_gefunden_1  \
0     NaN          NaN                           NaN   
1     NaN          NaN                           NaN   
2     NaN          NaN                           NaN   
3     NaN          NaN                           NaN   
4     NaN          NaN                           NaN   
5     NaN          NaN                           NaN   
6     NaN          NaN                           NaN   
7     NaN          NaN                           NaN   
8     NaN          NaN                           NaN   
9     NaN          NaN                           NaN   
10    NaN          NaN                           NaN   
11    NaN          NaN                           NaN   
12    NaN          NaN                           NaN   
13    NaN          NaN                           NaN   
14    NaN          NaN                           NaN   
15    NaN 

## Make an excel file

In [7]:
df.to_csv('Te2st.csv',',')

In [8]:
df.to_excel("dingo.xlsx")

In [ ]:
cv2.destroyAllWindows()